## Get the data
csv headings: id, created_at, source, original_text, clean_text, favorite_count, retweet_count, hashtags, trend <br>
hashtags format: strings with comma separated hashtags

In [173]:
#File paths
US_tweets_file = './Data/USTweets.csv'
UK_tweets_file = './Data/UKTweets.csv'

In [174]:
import csv

tweets = []
hashtags = [] #list of lists of hashtags e.g. hashtags[0] = ["hashtag1", "hashtag2"]
hashtags_strings = [] #list of hashtags string e.g. hashtags[0] = ["hashtag1, hashtag2"]


UK data

In [175]:
with open(UK_tweets_file) as data_file:
    data = csv.reader(data_file)
    next(data) #To skip the headings
    for row in data:
        tweets.append(row[4])
        hashtags.append(row[7].split(", "))
        hashtags_strings.append(row[7])

US data

In [176]:
# with open(US_tweets_file) as data_file:
#     data = csv.reader(data_file)
#     next(data) #To skip the headings
#     for row in data:
#         tweets.append(row[4])
#         hashtags.append(row[7].split(", "))
#         hashtags_strings.append(row[7])

## Initialize the tokenizers
Will use a specialized tokenizer for the hashtags because we need to encode all the hashtags. It also does not matter if the encoding of the tweets match the encoding of the hashtags.

In [177]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

In [178]:
hashtags_tokenizer = Tokenizer(oov_token="<OOV>")
hashtags_tokenizer.fit_on_texts(hashtags_strings)
hashtags_word_index = hashtags_tokenizer.word_index
hashtags_index_word = hashtags_tokenizer.index_word

In [179]:
print(f'There are {len(tweets)} tweets, ')
print(f'the tweets contain {len(tweets_tokenizer.word_index)} different words.')
print(f'There are {len(hashtags_tokenizer.word_index)} different hashtags')
print('Here are the tokenized hashtags')
print(hashtags_word_index)

There are 15006 tweets, 
the tweets contain 18029 different words.
There are 4403 different hashtags
Here are the tokenized hashtags
{'<OOV>': 1, 'pakvnz': 2, 'emergencyalert': 3, 'newtot': 4, 'universitychallenge': 5, 'imaceleb': 6, 'forgotten80s': 7, 'mastermind': 8, 'malpractice': 9, 'nufc': 10, 'panorama': 11, 'showman': 12, 'thfc': 13, 'bouwhu': 14, 'barçaatleti': 15, 'northernlights': 16, 'aurora': 17, 'spurs': 18, 'pakvsnz': 19, 'coys': 20, 'pakistancricket': 21, 'stgeorgesday': 22, 'rizwan': 23, 'jungkook': 24, 'derrenbrown': 25, 'cricketmubarak': 26, 'davisgarcia': 27, 'mufc': 28, 'auroraborealis': 29, 'tottenham': 30, 'nzvspak': 31, 'shaheenshahafridi': 32, 'nzvpak': 33, 'levyout': 34, 'doctorwho': 35, 'cricket': 36, 'earthday': 37, 'emergencyalerts': 38, 'swfc': 39, 'dcfc': 40, 'babarazam': 41, 'tuckercarlson': 42, 'premierleague': 43, 'avfc': 44, 'cfc': 45, 'chapman': 46, 'enicout': 47, 'pakistan': 48, 'facup': 49, 'newcastle': 50, 'barcaatleti': 51, 'pinkchella': 52, 'donb

## Prepare the pre-trained embeddings

In [247]:
import pickle
import numpy as np

num_tokens = len(tweets_word_index) + 1
embedding_dim = 300
hits = 0
misses = 0
missed_words = set()

embedding_matrix = np.zeros((num_tokens, embedding_dim))
with open('../Hashtag-recomender-embeddings/embeddings_index_object.pkl', 'rb') as embeddings_file:
    embeddings_index = pickle.load(embeddings_file)
    for word, i in tweets_word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            if embedding_vector.shape == (300,):
                embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
            missed_words.add(word)

In [249]:
print(f'hits:{hits}, misses: {len(missed_words)}')
print(f'missed words: {missed_words}')

hits:13982, misses: 4047
missed words: {'phonepic', 'nhy', 'dognews', 'kagwirawoupdates', 'emiratesfacup', 'freejulianassange', 'bbcpanorama', 'wawaw', 'kisikabhaikisikijaan', 'caalax', 'writerslift', 'sunvslions', 'dropsofgod', 'armstro', 'sawayama', 'lizplay', 'twatters', 'alcaraz', 'rtitbot', 'prizepick', 'forbesunder', 'pepedoge', 'mankindpharma', 'bristolcity', 'emergencypreparedness', 'loonknight', 'undersummit', 'jaaty', 'starmeritetory', 'htaa', 'racistmastermind', 'ninersfootball', 'auroraborealisuk', 'jumwa', 'rcbvrr', 'shaheeeen', 'elonbuyfoxnews', 'ethereums', 'nikonz', 'wewantyoutostay', 'fraserburgh', 'winsports', 'indieapril', 'earthda', 'mffl', 'tintoretto', 'chokistan', 'pahlavis', 'sofialert', 'midnor', 'yoursaracens', 'numbercat', 'notgoingtobelievethisbut', 'webridgexbambam', 'kanagajnr', 'mutualaid', 'outsinging', 'sunte', 'stoppate', 'harmen', 'bcafc', 'harrisonbarnes', 'bristolrovers', 'sacramentoproud', 'compartilhe', 'chapath', 'makeadifference', 'savingsyriasc

## Create the sequences and pad them and one multi-hot encode the hashtags
Will use a binary vector to encode the hashtags to the model can categorize the tweets. e.g. hashtags[0] = [tag1, tag2], and tag1 has encoding of 1 and tag2 has encoding 2, then the binary vector wil be [0 1 1 0 0 ... no_of_different_hashtags]

In [222]:
sequence_length = 15

from keras.utils import pad_sequences
tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
hashtags_sequences = hashtags_tokenizer.texts_to_sequences(hashtags)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

In [223]:
from keras.utils import to_categorical
import tensorflow as tf

no_of_different_hashtags = len(hashtags_word_index) + 1
no_of_hashtags = len(hashtags_sequences)

encoded_hashtags = np.zeros((no_of_hashtags, no_of_different_hashtags))

for i, hashtags_indices in enumerate(hashtags_sequences):
    encoded_hashtags[i][hashtags_indices] = 1


encoded_hashtags = np.array(encoded_hashtags)


## Split the data

In [224]:
training_split = 0.8
training_tweets_count = int(0.8 * len(tweets_sequences_padded))

In [225]:
train_data = tweets_sequences_padded[0:training_tweets_count]
train_labels = encoded_hashtags[0:training_tweets_count]
test_data = tweets_sequences_padded[training_tweets_count:]
test_labels = encoded_hashtags[training_tweets_count:]

print(f'we have {len(train_data)} tweets for training and {len(test_data)} for testing')

we have 12004 tweets for training and 3002 for testing


## Build the model

In [226]:
#hyperparameters
embedding_dimensions = 300
lstm_units = 128
dropout_value = 0.2
conv_filters = 64
conv_kernel_size = 5

In [238]:
#learning rate callback
def lr_schedule(epoch):
    lr = 0.001
    if epoch > 5:
        lr = 0.0001
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [239]:
from keras import initializers

no_of_tweets_words = len(tweets_word_index) + 1

hashtag_recommender_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(no_of_tweets_words,
                            embedding_dimensions,
                            input_length=sequence_length,
                            embeddings_initializer=initializers.Constant(embedding_matrix),
                            trainable=False),
    # tf.keras.layers.Conv1D(conv_filters, conv_kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_value),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
    # tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_different_hashtags, activation='softmax')
])

hashtag_recommender_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy'],
)

hashtag_recommender_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 15, 300)           5409000   
                                                                 
 dropout_37 (Dropout)        (None, 15, 300)           0         
                                                                 
 bidirectional_22 (Bidirecti  (None, 256)              439296    
 onal)                                                           
                                                                 
 dropout_38 (Dropout)        (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 4404)              1131828   
                                                                 
Total params: 6,980,124
Trainable params: 1,571,124
Non-trainable params: 5,409,000
___________________________________

## Train the model

In [240]:
epochs = 15
hashtag_recommender_model.fit(train_data, train_labels, epochs=epochs, validation_data=(test_data, test_labels), callbacks=[lr_scheduler])

Epoch 1/15
376/376 [==============================] - 18s 40ms/step - loss: 9.1275 - accuracy: 0.3297 - val_loss: 8.7489 - val_accuracy: 0.4620 - lr: 0.0010
Epoch 2/15
376/376 [==============================] - 14s 38ms/step - loss: 9.1289 - accuracy: 0.4778 - val_loss: 10.7730 - val_accuracy: 0.5163 - lr: 0.0010
Epoch 3/15
376/376 [==============================] - 14s 38ms/step - loss: 10.9286 - accuracy: 0.5070 - val_loss: 13.6183 - val_accuracy: 0.5260 - lr: 0.0010
Epoch 4/15
376/376 [==============================] - 14s 38ms/step - loss: 12.3220 - accuracy: 0.5238 - val_loss: 16.8834 - val_accuracy: 0.5440 - lr: 0.0010
Epoch 5/15
376/376 [==============================] - 15s 41ms/step - loss: 14.2630 - accuracy: 0.5337 - val_loss: 19.1499 - val_accuracy: 0.5430 - lr: 0.0010
Epoch 6/15
376/376 [==============================] - 15s 40ms/step - loss: 16.1172 - accuracy: 0.5423 - val_loss: 24.4162 - val_accuracy: 0.5613 - lr: 0.0010
Epoch 7/15
376/376 [=============================

## Get hashtags!!

In [232]:
def predict(tweet, tweet_tokenizer, hashtag_tokenizer, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    hashtag_indices = np.argsort(prediction, axis=-1)[0][-3:]
    return [hashtag_tokenizer.index_word[hashtag_index] for hashtag_index in hashtag_indices]


In [242]:
print(predict("GET THOSE FLAGS READY FOLKS AND BE PROUD OF WHO YOU ARE", tweets_tokenizer, hashtags_tokenizer, sequence_length, hashtag_recommender_model))

1/1 [==============================] - 0s 16ms/step
['forgotten80s', 'universitychallenge', 'emergencyalert']
